# Paper Count

How many papers did a journal publish?

In [1]:
import os

import pandas as pd
from sqlalchemy import create_engine, engine, MetaData, Table, select, and_, func
from snowflake.sqlalchemy import URL
from dotenv import load_dotenv, find_dotenv
from ipywidgets import Dropdown, HTML, IntSlider

In [2]:
# Get database credentials

load_dotenv(find_dotenv())

SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_ROLE = os.getenv("SNOWFLAKE_ROLE")
SNOWFLAKE_DATABASE = "PROD_EDW"
SNOWFLAKE_SCHEMA = "EBAC"
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")

In [3]:
# Create database engine

engine = create_engine(
    URL(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        role=SNOWFLAKE_ROLE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        warehouse=SNOWFLAKE_WAREHOUSE,
    )
)

In [4]:
# Connect and reflect tables

conn = engine.connect()
metadata = MetaData()

articles = Table("DW_ARTICLE_EXTN", metadata, autoload=True, autoload_with=engine)

In [5]:
# Get list of Wiley journals

stmt = (
    select(
        [articles.c.full_source_title])
        .where(articles.c.publisher_group == "WILEY")
    .distinct()
    )

journal_names = pd.read_sql(stmt, conn)

journal_names = journal_names.sort_values(by="full_source_title")

journal_list = journal_names["full_source_title"].to_list()

In [6]:
# Create a dropdown widget from the list

journal_dropdown = Dropdown(options=journal_list, description="Journal:")

In [7]:
# Create a year slider widget

year_slider = IntSlider(value=2017, min=2011, max=2020, description="Year:")

In [8]:
def get_article_count(journal, year):
    """
    Asks Snowflake for how many articles were published by a journal in a year.
    """
    stmt = (
    select(
        [func.count(articles.c.article_id)])
        .where(
            and_(articles.c.full_source_title==journal,
                articles.c.year_published==year))
    )
    
    df = pd.read_sql(stmt, conn)
    
    return df.iloc[0,0]

In [9]:
# Set the starting output text.

article_count_text = HTML("Select a journal.")

def update_article_count_text(change):
    """Handler that is called whenever a journal dropdown or year slider trait changes."""
    number_articles = get_article_count(journal_dropdown.value, year_slider.value)
    article_count_text.value = f"{journal_dropdown.value} published {number_articles} articles in {year_slider.value}."

# Execute a callback whenever the Dropdown or Slider inputs change    
journal_dropdown.observe(update_article_count_text, 'value')
year_slider.observe(update_article_count_text, 'value')

In [10]:
# Show the controls

display(journal_dropdown)
display(year_slider)

Dropdown(description='Journal:', options=('11TH OESO WORLD CONFERENCE: REFLUX DISEASE', '12TH OESO WORLD CONFE…

IntSlider(value=2017, description='Year:', max=2020, min=2011)

In [11]:
# Show the text

article_count_text

HTML(value='Select a journal.')